# Configure & verify OSPF config on Cat8000v

### For this notebook, it has been very useful to work with Cisco's YANG Suite, which streamlines the process of working with RESTCONF and NETCONF, creating filters, navigating the hierarchy of modules, etc. To do so, first you have to import the YANG modules from the repository of the device you are working with. In this case, it can be found at https://github.com/YangModels/yang/tree/main/vendor/cisco/xe/17121

<img src="./images/1-yang-repository.png" alt="Yang repository import"/>

### Then, you have to create a YANG set with all the desired modules. In this case, all of the have been selected. However, only Cisco-IOS-XE-native is required for this notebook

<img src="./images/2-yang-module-set.png" alt="Module set definition"/>

### After this, we have to create the a profile for the device we will be working with. We can set the address, username, password, as well as the RESTCONF and NETCONF configuration parameters. By doing this, we can directly interact with the device and send requests using these two protocols in the same Cisco YANG Suite.

<img src="./images/3-device-profile.png" alt="Device profile configuration"/>

### In these last two screenshots, we can see how we are working with the NETCONF and RESTCONF protocols for this device.

<img src="./images/4-netconf-get-example.png" alt="Netconf XML example"/>

<img src="./images/5-restconf-endpoint-example.png" alt="Restconf endpoint example"/>

# Configure OSPF using NETCONF

In [1]:
import ncclient.manager
from ncclient.operations import TimeoutExpiredError
import xml.dom.minidom
import pandas as pd

Dictionary containing all the necessary information to connect to the CAT8Kv. Abstracted from the code in order to provide this script as a "module", therefore adapting it quickly to new devices.

In [2]:
host_info = {
    'ip': '10.10.20.48',
    'username': 'developer',
    'pwd': 'C1sco12345',
    'netconf_port': '830',
    'restconf_port': '443'
}

### Write sample OSPF configuration to spreadsheet

Sample OSPF configuration that will be installed on the device using RESTCONF. First, it will be saved to a spreadsheet. If we need to adjust the configuration in the spreadsheet manually to account for some changes, we can run the code from the cell where the configuration is read from the spreadsheet to make them effective.

In [3]:
ospf_sheet = pd.ExcelWriter('ospf_conf.xlsx', engine='xlsxwriter')

processes = [
    {
        'process_id': '1',
        'router_id': '10.10.20.48'
    },
    {
        'process_id': '5',
        'router_id': '10.10.20.148'
    }
]

networks = [
    {
        'process_id': '1',
        'network': '10.10.20.0',
        'wildmask': '0.0.0.255',
        'area': '0'
    },
    {
        'process_id': '1',
        'network': '10.10.30.0',
        'wildmask': '0.0.0.255',
        'area': '0'
    },
    {
        'process_id': '1',
        'network': '10.10.40.0',
        'wildmask': '0.0.0.255',
        'area': '1'
    },
    {
        'process_id': '1',
        'network': '10.10.50.0',
        'wildmask': '0.0.0.255',
        'area': '1'
    }
]

processes_df = pd.DataFrame(processes)
networks_df = pd.DataFrame(networks)

processes_df.to_excel(ospf_sheet, sheet_name='Processes', index=False)
networks_df.to_excel(ospf_sheet, sheet_name='Networks', index=False)

ospf_sheet.save()
print("*** Spreadsheet successfuly filled with the data! ***")

*** Spreadsheet successfuly filled with the data! ***


### Load OSPF configuration from spreadsheet

In [4]:
ospf_sheet = pd.ExcelFile("ospf_conf.xlsx", engine='openpyxl')

processes_df = ospf_sheet.parse("Processes")
networks_df = ospf_sheet.parse("Networks")

In [5]:
processes_df

,process_id,router_id
0,1,10.10.20.48
1,5,10.10.20.148


In [6]:
networks_df

,process_id,network,wildmask,area
0,1,10.10.20.0,0.0.0.255,0
1,1,10.10.30.0,0.0.0.255,0
2,1,10.10.40.0,0.0.0.255,1
3,1,10.10.50.0,0.0.0.255,1


Parse the data from the DataFrames in two lists: processes and networks. Each list contains other nested lists with all the relevant configuration.

In [7]:
processes = processes_df.to_dict('records')
networks = networks_df.to_dict('records')
processes = [{k: str(v) for k, v in process.items()} for process in processes]
networks = [{k: str(v) for k, v in network.items()} for network in networks]

In [8]:
processes = [list(process.values()) for process in processes]
networks = [list(network.values()) for network in networks]

In [9]:
processes

[['1', '10.10.20.48'], ['5', '10.10.20.148']]

In [10]:
networks

[['1', '10.10.20.0', '0.0.0.255', '0'],
 ['1', '10.10.30.0', '0.0.0.255', '0'],
 ['1', '10.10.40.0', '0.0.0.255', '1'],
 ['1', '10.10.50.0', '0.0.0.255', '1']]

### Create the XML-content to set the configuration

The strings ospf_process_conf and ospf_networks_conf contain the XML-content to configure a process and a router ID, and all the relevant networks to be advertised, respectively.

In [11]:
conf_header_start = "<config xmlns='urn:ietf:params:xml:ns:netconf:base:1.0'>"
conf_header_end = "</config>"

ospf_process_conf = '''
<native xmlns="http://cisco.com/ns/yang/Cisco-IOS-XE-native">
    <router>
        <router-ospf xmlns="http://cisco.com/ns/yang/Cisco-IOS-XE-ospf">
            <ospf>
                <process-id>
                    <id>{}</id>
                    <router-id>{}</router-id>
                </process-id>
            </ospf>
        </router-ospf>
    </router>
</native>
'''

ospf_networks_conf = '''
<native xmlns="http://cisco.com/ns/yang/Cisco-IOS-XE-native">
    <router>
        <router-ospf xmlns="http://cisco.com/ns/yang/Cisco-IOS-XE-ospf">
            <ospf>
                <process-id>
                    <id>{}</id>
                    <network>
                        <ip>{}</ip>
                        <wildcard>{}</wildcard>
                        <area>{}</area>
                    </network>
                </process-id>
            </ospf>
        </router-ospf>
    </router>
</native>
'''

Generic function to update the configuration of a device based on an XML string and all the relevant parameters to be set.

In [12]:
def update_config(xml_config, config_parameters):
    try:
        device_connection = ncclient.manager.connect(
            host = host_info['ip'],
            username = host_info['username'],
            password = host_info['pwd'],
            port = host_info['netconf_port'],
            device_params= {'name': 'csr'},
            hostkey_verify=False
        )
        print("Connected to the device!")
    except:
        print("Failure...")
        
    configuration = conf_header_start
    for config_parameter in config_parameters:
        configuration += xml_config.format(*config_parameter)
    configuration += conf_header_end
    device_connection.edit_config(target='running', config=configuration)
    print("Config pushed successfuly!")

In [13]:
update_config(ospf_process_conf, processes)

Connected to the device!
Config pushed successfuly!


In [14]:
update_config(ospf_networks_conf, networks)

Connected to the device!
Config pushed successfuly!


# Verify configuration using RESTCONF

In [15]:
import requests
import json

Set the headers for the HTTP request, stating the content to be sent and received (JSON/yang-data).

In [16]:
headers = {'Content-Type': 'application/yang-data+json',
            'Accept': 'application/yang-data+json'} 

### Craft and send HTTP GET request to the appropriate endpoint

In [17]:
api_endpoint = "data/Cisco-IOS-XE-native:native/router/Cisco-IOS-XE-ospf:router-ospf/ospf/process-id"
url_begin = "https://{}:{}/restconf/"

In [18]:
url_begin = url_begin.format(host_info['ip'], host_info['restconf_port'])

In [19]:
url = url_begin + api_endpoint

Send the request to get the OSPF configuration and check if the response code is 200 (OK).

In [20]:
response = requests.get(url, auth=(host_info['username'], host_info['pwd']), headers=headers, verify=False)

if (response.status_code != 200):
    print("*** Something went wrong with the request! ***")

/Users/gerivives/opt/anaconda3/lib/python3.9/site-packages/urllib3/connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host '10.10.20.48'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


In [21]:
print(response.text)

{
  "Cisco-IOS-XE-ospf:process-id": [
    {
      "id": 1,
      "network": [
        {
          "ip": "10.10.20.0",
          "wildcard": "0.0.0.255",
          "area": 0
        },
        {
          "ip": "10.10.30.0",
          "wildcard": "0.0.0.255",
          "area": 0
        },
        {
          "ip": "10.10.40.0",
          "wildcard": "0.0.0.255",
          "area": 1
        },
        {
          "ip": "10.10.50.0",
          "wildcard": "0.0.0.255",
          "area": 1
        }
      ],
      "router-id": "10.10.20.48"
    },
    {
      "id": 5,
      "router-id": "10.10.20.148"
    }
  ]
}



### Parse the data to compare it with the spreadsheet

In [22]:
processes_get = []
networks_get = []

Parse all the data from the JSON response in the same format as the data read from the spreadsheet to compare it.

In [23]:
data = json.loads(response.text)
for process in data['Cisco-IOS-XE-ospf:process-id']:
    processes_get.append([str(process['id']), process['router-id']])
    if ('network' in process):
        for network in process['network']:
            networks_get.append([str(process['id']), network['ip'], network['wildcard'], str(network['area'])
            ])

In [24]:
processes_get

[['1', '10.10.20.48'], ['5', '10.10.20.148']]

In [25]:
processes

[['1', '10.10.20.48'], ['5', '10.10.20.148']]

In [26]:
networks_get

[['1', '10.10.20.0', '0.0.0.255', '0'],
 ['1', '10.10.30.0', '0.0.0.255', '0'],
 ['1', '10.10.40.0', '0.0.0.255', '1'],
 ['1', '10.10.50.0', '0.0.0.255', '1']]

In [27]:
networks

[['1', '10.10.20.0', '0.0.0.255', '0'],
 ['1', '10.10.30.0', '0.0.0.255', '0'],
 ['1', '10.10.40.0', '0.0.0.255', '1'],
 ['1', '10.10.50.0', '0.0.0.255', '1']]

In [28]:
if (processes_get == processes):
    print("*** The processes and/or router IDs are the same! ***")

*** The processes and/or router IDs are the same! ***


In [29]:
if (networks_get == networks):
    print("*** The networks and/or areas for the processes are the same! ***")

*** The networks and/or areas for the processes are the same! ***
